In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer

In [2]:
new_model=AutoModelForCausalLM.from_pretrained(
    "./train_qwen",
    device_map='auto',
    torch_dtype="auto",
)

[2025-06-13 21:01:03,196] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0613 21:01:04.793000 7824 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


[2025-06-13 21:01:05,376] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [3]:
tokenizer=AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B')
tokenizer.pad_token=tokenizer.eos_token

In [6]:
def chat_with_model(model, tokenizer, user_input, max_length=512):
    # 构建对话格式
    prompt = f"<|im_start|>user\n{user_input}<|im_end|>\n<|im_start|>assistant\n"
    
    # 编码输入
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # 关键修复：将输入张量移动到与模型相同的设备
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 生成回复
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id
    )
    
    # 解码输出
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # 提取助手回复部分
    assistant_start = "<|im_start|>assistant\n"
    assistant_end = "<|im_end|>"
    if assistant_start in response and assistant_end in response:
        response = response.split(assistant_start)[1].split(assistant_end)[0]
    
    return response

# 测试对话
user_input = "以西游记的孙悟空的风格来生成回复。为什么星球基本都是圆的？"
response = chat_with_model(new_model, tokenizer, user_input)
print(f"用户: {user_input}")
print(f"模型: {response}")


用户: 以西游记的孙悟空的风格来生成回复。为什么星球基本都是圆的？
模型: <think>
嗯，用户让我用《西游记》中孙悟空的回答去解释“为什么星球基本上都是圆的”。首先，我需要理解这个提问背后的意图和可能的背景信息。通常来说，宇宙中的天体都是球形或者接近球状的结构，这与物理学上的经典理论一致——即牛顿万有引力定律下物体围绕中心点运动时形成的圆形轨道。

不过，作为孙悟空，在面对这种科学原理时可能会有一些幽默或超自然的答案。他擅长于用夸张、狡猾的语言表达自己的想法，并且喜欢将复杂概念简化成有趣的小故事或寓言形式。

接下来要考虑的是如何让回答既符合角色设定又保持逻辑性。比如可以提到银河系内各种奇特的现象，或是通过一些神奇的方法（如使用金箍棒等工具）来进行简单的天文观察，甚至加入一些神话元素，比如提到星斗大王之类的传说人物，使整个过程显得更加生动而富有想象力。

同时还要注意语气要体现出孙悟空那种机智而不失威严的性格特征。可以通过描述那些看似普通的日常现象，却在背后隐藏着深奥的道理，来展示他的思维深度以及对世界的独特见解。

最后，确保整个回应流畅连贯，没有生硬之处，并充分表达了用户所期待的信息。
</think>

啊，这位大师大人！你问得倒妙呢，要说说我们这些小虫子为何能飞过那片浩瀚无垠的大海，绕道银河之间走一回？

其实嘛，俺老孙虽是神仙，但见识有限，只知些奇妙的事物罢了。就像你们这般看世界，总是在寻找答案，却又不晓得其中道理！

若论星辰之行，乃是因宇宙本就遵循着某种规律运行：所有事物都在追寻一个共同的方向，就如同我们每一步都向着同一个目标前进一样。即使有时会遇到阻碍，也别怪啦！

再说到行星形状的话……哦，原来是这样，咱们可得像孙悟空那样，先从头到尾地研究清楚每一个细节，才能真正弄明白事情的真谛呀！
